## Capstone Project: Dataset Name - Exploratory Data Analysis ##


**Problem Statement:**  

**Stakeholders:** 
* Group 1
* Group 2
* Group 3

## Part A. Import Cleaned Dataset ##

In [1]:
# Import all libraries #
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import csv
from sklearn.utils import shuffle
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Authorization #
__author__ = "Taesun Yoo"
__email__ = "yoots1988@gmail.com"

In [2]:
# --- 1. Write Out List of Functions --- #
def load_file(file):
    '''load input CSVs as a dataframe'''
    return pd.read_csv(file, encoding='latin1')


def convert_dt_as_date(df, var_name, date_format):
    '''convert the variable as specified date format'''
    df[var_name] = pd.to_datetime(df[var_name], format=date_format)
    return df[var_name]


def convert_dt_as_custom(df, var_name, dt_type):
    '''convert datatype on selected variables'''
    df[var_name] = df[var_name].astype(dt_type)
    return df[var_name]


def convert_dt_to_cat(df):
    '''convert data type to category'''
    for col in df.columns:
        if df[col].dtype.name == 'object':
            df[col] = df[col].astype('category')


def eda_encode_cat_var(df, col, num_var):
    '''encode the cat. variables by mean of a num. variable by each cat'''
    cat_dict={}
    cats = df[col].cat.categories.tolist()
    for cat in cats:
        cat_dict[cat] = df[df[col] == cat][num_var].mean()
    df[col] = df[col].map(cat_dict)


#def pandas_qcut(df, new_var, old_var, q):
#    ''' apply pd.qcut for equal size binning '''
#    df[new_var] = pd.qcut(df[old_var], q=q, duplicates='drop',
#                          precision=0, labels=None)
#    return df[new_var]


def estimate_width(df, var_name, interval_size):
    ''' estimate the width of continuous variable '''
    width = (max(df[var_name]) - min(df[var_name]))/interval_size
    df_estimate = pd.DataFrame([[min(df[var_name])-1, max(df[var_name])+1, round(width)]])
    df_estimate.columns = ['min', 'max', 'width']
    return df_estimate


def pandas_cut(df, new_var, old_var, bins, labels=None):
    ''' apply pd.cut for custom size binning '''
    df[new_var] = pd.cut(df[old_var], bins=bins, 
                          labels=labels, right=False)
    return df[new_var]


def check_value_counts(df, var_name):
    '''return grouped value counts'''
    grouped_counts = df[var_name].value_counts()
    return grouped_counts


def feature_replacement(X):
    ''' replace missing values based on specific data type of a column '''
    for col in X.columns:
        if X[col].dtype.name == 'object':
            mode = X[col].mode().iloc[0]
            X[col] = X[col].fillna(mode)
        elif X[col].dtype.name == 'float64':
            mean = X[col].mean()
            X[col] = X[col].fillna(mean)
        else:
            X[col].dtype.name == 'int64'
            median = X[col].median()
            X[col] = X[col].fillna(median)
            

def eda_missing(df):
    '''compute missing % on each var'''
    df_missing = pd.DataFrame(df.isnull().sum(), columns=['count'])
    df_missing['pct'] = (df_missing['count']/len(df)) * 100
    return df_missing


def eda_stat_num(df):
    ''' perform eda for numerical features '''
    df_stat_num = df.describe().T
    df_stat_num = df_stat_num[['count', 'min', 'mean', 'max', 'std', '25%', '50%', '75%']]
    df_stat_num = pd.DataFrame(df_stat_num)
    return df_stat_num


def eda_stat_cat(df):
    ''' perform eda for categorical features '''
    df_stat_cat = df.describe(include='category').T
    df_stat_cat = pd.DataFrame(df_stat_cat)
    return df_stat_cat


def eda_outliers(df):
    '''check outliers using the IQR method'''
    df['IQR'] = df['75%'] - df['25%']
    df['LB']  = df['25%'] - 1.5*df['IQR']
    df['UB']  = df['75%'] + 1.5*df['IQR']
    df = df.drop(['count','std','mean','25%','50%','75%','IQR'], axis=1)
    return df


def eda_agg_df_var(df, cat_var, kpi_dict):
    '''compute aggregated dataframe to calculate the KPIs'''
    df_agg = df.groupby(by=cat_var).agg(kpi_dict)
    return df_agg


def eda_grouped_df_var(df, cat_var):
    '''create a grouped dataframe by categorical variable'''
    df_grp = pd.DataFrame(df.groupby([cat_var])[cat_var].count())
    df_grp.columns = ['count']
    return df_grp


def plot_hist(df, var_1):
    '''plot a histogram'''
    plt.figure()
    print("skenewss is:", df[var_1].skew())
    df[var_1].hist(color='green')
    plt.title('Histogram of ' + var_1)
    plt.xlabel(var_1)
    plt.ylabel('patients')
    plt.show()


def plot_bar_chart(df, var_name_1):
    '''plot a bar chart'''
    plt.figure()
    var_count_1 = df[var_name_1].value_counts()
    sns.barplot(var_count_1.index,  var_count_1.values, alpha=0.9)
    plt.title('Frequency chart of ' + var_name_1)
    plt.ylabel('patients')
    plt.xlabel(var_name_1)
    plt.show()

    
def plot_freq_chart(x,y,df,order):
    '''plot a frequency chart'''
    plt.figure(figsize=(8,8))
    sns.countplot(x=x, hue=y, data=df, order=order)
    plt.title('Bar chart: ' + x + ' of client group labels', fontsize=20)
    plt.xticks(rotation=270, fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel(x, fontsize=12)
    plt.ylabel('patients', fontsize=12)
    plt.legend(loc='upper right', fontsize=20)
    plt.show()


def plot_pie_chart(df_1, var_name_1,
                   df_2, var_name_2):
    '''plot a pie chart of specified variables'''
    plt.figure(figsize=(15,15))
    # Sub-plot 1:
    plt.subplot(1,2,1)
    plt.pie(df_1, autopct='%.0f%%', wedgeprops={'edgecolor':'white'},
            textprops={'fontsize':15})
    plt.title('Pie Chart of '+ var_name_1)
    plt.legend(labels = df_1.index, loc='upper right')
    # Sub-plot 2:
    plt.subplot(1,2,2)
    plt.pie(df_2, autopct='%.0f%%', wedgeprops={'edgecolor':'white'},
            textprops={'fontsize':15})
    plt.title('Pie Chart of '+ var_name_2)
    plt.legend(labels = df_2.index, loc='upper right')
    plt.show()
    

def plot_box(df, num_var_1, cat_var_1,
             num_var_2, cat_var_2, 
             num_var_3, cat_var_3, hue=None):
    '''plot a box-whisker of specified variables'''
    plt.figure(figsize=(15,15))
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    # Sub-plot 1:
    plt.subplot(1,3,1)
    df.sort_values(by=[num_var_1], inplace=True)
    sns.set(style='whitegrid')
    sns.boxplot(cat_var_1, num_var_1, hue, df)
    plt.title('Box plot of ' + num_var_1 + ' by ' + cat_var_1)
    plt.xticks(rotation=270, fontsize=10)
    # Sub-plot 2:
    plt.subplot(1,3,2)
    df.sort_values(by=[num_var_2], inplace=True)
    sns.set(style='whitegrid')
    sns.boxplot(cat_var_2, num_var_2, hue, df)
    plt.title('Box plot of ' + num_var_2 + ' by ' + cat_var_2)
    plt.xticks(rotation=270, fontsize=10)
    # Sub-plot 3:
    plt.subplot(1,3,3)
    df.sort_values(by=[num_var_3], inplace=True)
    sns.set(style='whitegrid')
    sns.boxplot(cat_var_3, num_var_3, hue, df)
    plt.title('Box plot of ' + num_var_3 + ' by ' + cat_var_3)
    plt.xticks(rotation=270, fontsize=10)    

    
def plot_crosstab(df, cat_var_1, cat_var_2):
    '''plot a crosstab of two categorical variables'''
    table = pd.crosstab(df[cat_var_1], df[cat_var_2])
    return table


def plot_corr_matrix(df, list_vars):
    ''' plot a correlation matrix '''
    corr = df[list_vars].corr()
    # Create a mask
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(corr)] = True
    plt.figure(figsize=(12,12))
    sns.heatmap(corr, mask=mask, square=True, linewidths = .5,
                cmap=sns.diverging_palette(220,10,as_cmap=True),
                vmin = -1, vmax = 1, fmt=".2f",
                annot=True, annot_kws={'size':15})
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.show()


def plot_scatter(df, var_1, var_2, color, factor=None):
    '''Scatter plot of two continuous numeric features'''
    plt.figure(figsize=(8,8))
    plt.scatter(df[var_1], df[var_2], color=color)
    plt.title('Relationship between '+ var_1 + ' and ' + var_2)
    plt.xlabel(var_1)
    plt.ylabel(var_2)
    plt.show()

    
def compute_pearson_r(df, var_x, var_y):
    '''compute Pearson r correlation'''
    corr_mat = np.corrcoef(df[var_x],df[var_y])
    return corr_mat[0, 1]


def plot_linear_reg(df, var_x, var_y, 
                    pearson_r, color, label):
    '''plot a pair of linear regressions'''
    plt.figure(figsize=(10,10))
    plt.plot(df[var_x], df[var_y], 'r--', label='pearson_r =%.2f' % pearson_r,
             marker='.', linestyle='none', color=color)
    plt.margins(0.02)
    plt.legend(loc='upper left')
    plt.xlabel(var_x)
    plt.ylabel(var_y)
    plt.title(var_x + ' vs. ' + var_y + ' by ' + label)
    # Fit linear regression:
    a,b = np.polyfit(df[var_x], df[var_y], 1)
    x = np.array([min(df[var_x]), max(df[var_x])])
    y = a*x + b
    plt.plot(x,y)

In [3]:
# --- 2. Load the data --- #
# define input CSVs:
if __name__ == '__main__':
    eda_file = 'insurance_train.csv'

# load data:
df_eda = load_file(eda_file)

# map column names to lowercase:
df_eda.columns = map(str.lower, df_eda.columns)

# check data types:
df_eda.info()

FileNotFoundError: [Errno 2] File b'insurance_train.csv' does not exist: b'insurance_train.csv'

In [ ]:
# define variables:
#vars_cat = list(df_eda.select_dtypes(include='object').columns)
#vars_num_disc = list(df_eda.select_dtypes(include='int64'))
#vars_num_cont = list(df_eda.select_dtypes(include='float64'))

# concatenate a list:
#vars_num = vars_num_disc + vars_num_cont

# delete a list of unwanted variables:
#unwanted_list = {''}
#vars_num = [var for var in vars_num if var not in unwanted_list]

#var_label = 
#var_id = 

## Part B. Exploratory Data Analysis ##

### Dataset Name: Training Set ###
Data exploration is conducted on a cleaned training set. The main goal of this phase is to explore any interesting relationships among features and identify which features are good predictors on poverty rate predictions.

Following set of questions are asked:
1. Can I count something interesting?
2. Can I find some trends (increase or decrease and any anomalies)?
3. Can I plot a bar chart or a histogram?
4. Can I make a scatter plot?

These set of guiding questions will help us to explore any insights and tell a compelling story about the US poverty dataset.

In [ ]:
# compute top 10 rows:

# check duplicates:

In [ ]:
# create a missing % dataframe:


### Summary: Missing Values % ###
Above table shows that missing value % on each feature. app_underwriting_score misses about 3.7%.

In [ ]:
# compute feature impuation by data types:


In [ ]:
# create a missing % dataframe:


In [ ]:
# convert data type: object to category


In [ ]:
# numerical features:


### Summary: Numerical Features ###
* age: mean is around 51
* premium: mean is around ~ 11K
* income: mean is around ~209K
* % premium paid by cash credit: mean is 31%

In [ ]:
# categorical features:


### Summary: Categorical Features ###
* sourcing channel: majority of clients are sourced from channel A
* residence area type: majority of clients are from Urban

In [ ]:
# ---4 detect outliers ---
# create a dataframe for IQR:


# check outliers:
# lower bounds (LBs)

# upper bounds (UBs)

In [ ]:
# ---5 aggregate dataframe and compute KPIs ---
# Define the dictionary for KPIs:


# Print a summary KPI table by Renewal Status:


### Summary: Renewal Status ###
Comparison between non-renewed vs. renewed client groups:
* mean age of non-renewed client group is younger and % of premium paid by cash credit is higher. 
* Also average income and average premium paid amount are lower than renewed client group.

### Uni-variate: Histogram Plots ###

In [ ]:
# plot a histogram:    


### Summary: Histograms ###
* Age: close to normal distribution with mean is around 51.
* Premium: positively skewed with majority of clients paid premium lower than 15K.
* % Premium Paid by Cash Credit: positively skewed with majority of clients less than 15%
* Underwriting Score: negatively skewed with majority of clients above score 99.

### Uni-variate: Bar Charts ###

In [ ]:
# plot a bar chart:


### Summary: Bar Charts ###
* Sourcing Channel: majority of clients sourced from Channel A followed by B, C, D and E.
* Residence Area Type: majority of clients reside in Urban area.

### Frequency Charts ###

In [ ]:
# plot a frequency chart:


### Summary: Frequency Chart 1 ###
* Age Group: majority of client age group is around 40 to 60s.
* Income Group: majority of client income group is around 204K.
* Premium Group: majority of client premium paid group is around 16K.

### Uni-variate: Pie Charts ###

In [ ]:
# grouped dataframe by a categorical feature:


# Plot pie chart(s) by categorical features:


### Summary: Pie Chart ###
* Renewal Status: 94 % of clients renewed their policies.
* Residence Type: 60% of clients reside in Urban area.
* Age Group: about 50% of clients from 40-60s followed by 60-80s, 20-40s and +80.
* Sourcing Channel: 54% of clients from Channel A followed by B, C, D, E 

### Bi-variate: Box-Whisker Plots ###

In [ ]:
# plot a box-whisker:    


### Summary: Box-Whisker Plot ###
* Age: mean age of renewed client group is higher than non-renewed group.
* Premium: mean premiumd paid amout of renewed client group is higher than non-renewed group.
* % Premiumd Paid by Cash Credit: non-renewed group has higher usage of cash credit than renewed group.

### Bi-variate: Correlation Matrix Plot ###

In [ ]:
# create a dataframe:

# check data type:

# convert data type: object to category


# encode categorical variables using specified numeric feature:


# Create a list of variables:


# Delete a list of unwanted variables:


# plot a correlation matrix:


### Summary: Correlation Matrix Plot ###
1. Renewal status shows negative moderate correlation with count premium paid 6-12 months late (r=-0.28)
2. Renewal status shows negative weak correlation with % premium paid by cash credit (r=-0.24).
3. Premium shows positive moderate correlation with income (r=0.30)
4. Age shows negative weak correlation with % premium paid by cash credit (r=-0.26)

### Bi-variate: Cross Tabulation ###

In [ ]:
# plot a cross-tabulation:


### Summary: Cross Tabulation ###
1. renewal and age group: shows that age +80 group have the lowest # of clients in both groups. Majority of clients are from age 40-60s in both renewed and non-renewed client groups.
2. renewal and sourcing channel: shows that channel E group has the lowest # of clients in both groups. Majority of clients are from Channel A.

### Linear Regression Analyses ###

In [ ]:
#---- Plot a linear regression plot: w numerical variables ----#
# Compute Pearson r for combination of X and Y:


In [ ]:
# Plot a linear regression analysis:


### Summary: Linear Regression ###